In [ ]:
!pip install mutagen wikipedia-api diffusers torch transformers accelerate


In [1]:
# Install necessary packages (uncomment and run if not already installed)
# !pip install mutagen wikipedia-api diffusers torch transformers accelerate

# Import libraries
import os
import mutagen
import requests
import wikipediaapi
from mutagen.mp3 import MP3
from mutagen.flac import FLAC
from mutagen.easyid3 import EasyID3
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Select an audio file using a file dialog (or hardcode a path)
def choose_audio_file():
    root = Tk()
    root.withdraw()
    file_path = askopenfilename(filetypes=[("Audio Files", "*.mp3 *.flac")])
    return file_path

# Extract metadata from the uploaded song
def extract_metadata(filename):
    try:
        if filename.endswith(".mp3"):
            audio = MP3(filename, ID3=EasyID3)
        elif filename.endswith(".flac"):
            audio = FLAC(filename)
        else:
            return "❌ Unsupported file format."

        metadata = {
            "title": audio.get("title", ["Unknown"])[0],
            "artist": audio.get("artist", ["Unknown"])[0],
            "album": audio.get("album", ["Unknown"])[0],
            "year": audio.get("date", ["Unknown"])[0],
            "genre": audio.get("genre", ["Unknown"])[0],
        }
        return metadata
    except Exception as e:
        return f"⚠ Error extracting metadata: {str(e)}"

# Fetch historical information from Wikipedia
def get_wikipedia_info(year):
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        user_agent="MyVSCodeScript/1.0 (contact: your_email@example.com)"  # Update contact
    )
    page = wiki_wiki.page(str(year))
    if page.exists():
        return page.summary[:500]
    else:
        return f"No historical information found for the year {year}."

# Generate an AI image using Stable Diffusion
def generate_image(prompt):
    model_id = "CompVis/stable-diffusion-v1-4"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")

    image = pipe(prompt).images[0]
    image_path = "generated_image.png"
    image.save(image_path)
    return image_path

# Main logic
if __name__ == "__main__":
    filename = choose_audio_file()
    if not filename:
        print("❌ No file selected.")
    else:
        print("Selected File:", filename)

        # Metadata extraction
        song_metadata = extract_metadata(filename)
        print("\n🎵 Extracted Song Metadata:")
        print(song_metadata)

        # Historical info
        year = song_metadata.get("year", "Unknown")
        era_info = get_wikipedia_info(year)
        print("\n📜 Historical Information from Wikipedia:")
        print(era_info)

        # Image generation
        prompt = f"Visual representation of an actress dancing in {year}. {era_info}"
        image_path = generate_image(prompt)
        print("\n🖼 Generated Image Saved As:", image_path)

        # Optional: display image if using Jupyter
        Image.open(image_path).show()


ModuleNotFoundError: No module named 'mutagen'